In [2]:
import sys  
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
path_dir = r"/Users/../Volumes/Affiliations/"

In [3]:
import sys  
import pickle
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
import src.utils
from src.utils import import_dta, import_tracks_dta, successful_creators_followers,\
gen_active_relations, get_fan_interactions_per_week, calculate_avg_monthly_valence,\
gen_active_relations_prob, get_fan_interactions_per_week_prob, stripplot_prob,\
reaction_probability, follower_list, filter_quantile, sample_creators_music,\
gen_outbound_creators
import datetime
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mstats
pd.plotting.register_matplotlib_converters()
import os

import src.probability_utils
from src.probability_utils import volume_analysis, valence_analysis
import math
import matplotlib.ticker as plticker
from scipy.interpolate import make_interp_spline, BSpline
import seaborn as sns
from numpy import random
from tabulate import tabulate
from IPython.display import display
from tqdm import tqdm

In [4]:
def process_date(date):
    year = date[0:4]
    week = date[6:]
    day = "1"
    date = "{}-{}-1".format(year, week)
    dt = datetime.strptime(date, "%Y-%W-%w")
    return dt

In [5]:
path_dir = r"/Users/../Volumes/Raw/"
music_before = 30
low_success = 0.25
high_success = 0.75

In [6]:
#we will skip comments for now

#affiliations :follows
#favoritings :likes

tracks = import_tracks_dta(path_dir, "12sample_tracks.dta");
follows_sent = import_dta(path_dir, "12sample_affiliations_sent.dta");
comments_sent = import_dta(path_dir, "12sample_comments_made.dta");
shares_sent = import_dta(path_dir, "12sample_reposts_made.dta");
likes_sent = import_dta(path_dir, "12sample_favoritings_made.dta");
messages_sent = import_dta(path_dir, "12sample_messages_sent.dta");

user_info = import_dta(path_dir, "12sample_user_infos.dta");
user_info_2 = import_dta(path_dir, "12sample_1st_deg_user_infos.dta");
user_info = pd.concat([user_info, user_info_2]);
user_info.drop_duplicates(subset='user_id', keep='first', inplace=True, ignore_index=False);


########
follows_received = import_dta(path_dir, "12sample_affiliations_received.dta");
comments_received = import_dta(path_dir, "12sample_comments_received.dta");
shares_received = import_dta(path_dir, "12sample_reposts_received.dta");
likes_received = import_dta(path_dir, "12sample_favoritings_received.dta");
messages_received = import_dta(path_dir, "12sample_messages_received.dta");

########
follows = import_dta(path_dir, "12sample_affiliations.dta");
comments = import_dta(path_dir, "12sample_comments.dta");
shares = import_dta(path_dir, "12sample_reposts.dta");
likes = import_dta(path_dir, "12sample_favoritings.dta");
messages = import_dta(path_dir, "12sample_messages.dta");

tracks_first_month = sample_creators_music(tracks, music_before)
mask = tracks_first_month.track_available == 1
tracks_first_month = tracks_first_month[mask]

mask = follows_received.contact_id.isin(tracks_first_month.user_id)
follows_received_30 = follows_received[mask]
followers_per_creator_60 = successful_creators_followers(follows_received_30, 60, tracks_first_month)
followers_per_creator_60_static = successful_creators_followers(follows_received_30, 60, tracks_first_month, always_same_batches = True, 
                                                        perc1 = low_success, perc2 = high_success)

%%%%%%%%%% 12sample_tracks.dta %%%%%%%%%%
(56262, 7)
%%%%%%%%%% 12sample_affiliations_sent.dta %%%%%%%%%%
(800913, 3)
%%%%%%%%%% 12sample_comments_made.dta %%%%%%%%%%
(29258, 4)
%%%%%%%%%% 12sample_reposts_made.dta %%%%%%%%%%
(179329, 4)
%%%%%%%%%% 12sample_favoritings_made.dta %%%%%%%%%%
(527701, 4)
%%%%%%%%%% 12sample_messages_sent.dta %%%%%%%%%%
(11091, 3)
%%%%%%%%%% 12sample_user_infos.dta %%%%%%%%%%
(35000, 3)
%%%%%%%%%% 12sample_1st_deg_user_infos.dta %%%%%%%%%%
(670746, 3)
%%%%%%%%%% 12sample_affiliations_received.dta %%%%%%%%%%
(432503, 3)
%%%%%%%%%% 12sample_comments_received.dta %%%%%%%%%%
(21386, 4)
%%%%%%%%%% 12sample_reposts_received.dta %%%%%%%%%%
(83013, 4)
%%%%%%%%%% 12sample_favoritings_received.dta %%%%%%%%%%
(286903, 4)
%%%%%%%%%% 12sample_messages_received.dta %%%%%%%%%%
(17364, 3)
%%%%%%%%%% 12sample_affiliations.dta %%%%%%%%%%
(1233416, 3)
%%%%%%%%%% 12sample_comments.dta %%%%%%%%%%
(50644, 4)
%%%%%%%%%% 12sample_reposts.dta %%%%%%%%%%
(262342, 4)
%%%%%%%%%% 12sam

In [1]:
d = {}
for i in range(6):
   d[str(i)] = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/{}.pkl'.format(i)) 

NameError: name 'pd' is not defined

In [ ]:
data = pd.concat([d['0'], d['1'], d['2'], d['3'], d['4'], d['5']])

In [10]:
data['created_at'] = pd.to_datetime(data['created_at'])

In [11]:
data.set_index('created_at', inplace = True)

In [12]:
#outdegree = data.groupby('sender_id').agg(following=('contact_id', pd.Series.nunique))
#outdegree_info = user_info_2.merge(outdegree, left_on = 'user_id', right_on = 'sender_id', how= 'left')
#outdegree_info.loc[outdegree_info.following.isnull(), 'following'] = 0 

In [13]:
#outdegree_info['decile'] = pd.qcut(outdegree_info['following'], 10, labels=False)

In [14]:
#outdegree_info.head(10)

In [15]:
#outdegree_info['decile'].value_counts()

In [16]:
actions_sent = gen_outbound_creators(follows_sent, shares_sent, likes_sent, comments_sent, tracks,
                                     messages_sent = messages_sent, filter_creators = True)
mask = actions_sent.user_id.isin(tracks_first_month.user_id.unique())
actions_sent = actions_sent[mask]

In [17]:
#filter only outbound activities before the follow is received
print('Obtaining fanbase...')
follows_received.columns = ['fan_id', 'user_id', 'date_sent']
followers = follower_list(follows_received)
actions_sent = actions_sent.merge(followers, right_on = ['user_id', 'fan_id'],
                                      left_on = ['user_id', 'fan_id'], how = 'left')

Obtaining fanbase...


In [18]:
mask = (actions_sent.date_sent < actions_sent.follower_since) | (actions_sent.follower_since.isnull())
actions_sent_non_followers =  actions_sent[mask]
actions_sent_non_followers['week_yr_date'] = actions_sent_non_followers.week_yr.apply(lambda x: process_date(x))

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_90456/3898105865.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_non_followers['week_yr_date'] = actions_sent_non_followers.week_yr.apply(lambda x: process_date(x))


In [19]:
actions_sent_non_followers = actions_sent_non_followers.merge(outdegree_info, left_on = 'fan_id', right_on = 'user_id', how = 'inner')

NameError: name 'outdegree_info' is not defined

In [ ]:
actions_sent_non_followers.groupby('decile', as_index = False).size()

In [ ]:
import datetime
# Target Creation
delta = datetime.timedelta(days = 7)
mask = (actions_sent_non_followers['follower_since'] <(actions_sent_non_followers['date_sent'] + delta))

actions_sent_non_followers.loc[mask, 'reward'] = 1
mask = actions_sent_non_followers['reward'].isnull()
actions_sent_non_followers.loc[mask, 'reward'] = 0

# MAB

In [20]:
def create_trials_rewards(df):
    '''
    Creates reward counts, trial counts and keys to be used in the thompson sampling function
    '''
    Sent = df.groupby('outbound_activity', as_index = True).size().to_frame(name='sent')\
    .reset_index()
    Reward = df.groupby('outbound_activity', as_index = True)\
    .agg(b_Reward = ('reward', 'sum')).reset_index()


    keys  = Sent.outbound_activity

    return np.array(Sent.sent), np.array(Reward.b_Reward), keys

In [21]:
def update_1_step(Sent, Reward):
    
    alpha = Reward + 1 
    beta = (Sent-Reward)+1

    mean = alpha/(alpha+beta)
    variance = (alpha*beta)/((alpha+beta+1)*((alpha+beta)**2))

    return np.around(mean,3), np.around(np.sqrt(variance),3)

In [ ]:
def real_time(df, outdegree_df, interval):
    dataset_mean = pd.DataFrame({'trials':range(1,df.shape[0]+1), 
              '1': 0,
              '2': 0,
              '3': 0,
              '4': 0,
              '5': 0,
              '6': 0,
              '7': 0,
              '8': 0,
              '9': 0,
              '10': 0})
    
    dataset_std = pd.DataFrame({'trials':range(1,df.shape[0]+1), 
              '1': 0,
              '2': 0,
              '3': 0,
              '4': 0,
              '5': 0,
              '6': 0,
              '7': 0,
              '8': 0,
              '9': 0,
              '10': 0})
        
    experiment = {'1':{'Sent':0,'Reward':0}, 
             '2':{'Sent':0,'Reward':0},
             '3':{'Sent':0,'Reward':0},
             '4':{'Sent':0,'Reward':0},
             '5':{'Sent':0,'Reward':0}, 
             '6':{'Sent':0,'Reward':0},
             '7':{'Sent':0,'Reward':0},
             '8':{'Sent':0,'Reward':0},
             '9':{'Sent':0,'Reward':0},
             '10':{'Sent':0,'Reward':0}}

    date_l = min(df.date_sent)
    old_outdegree = outdegree_df.iloc[:
                  outdegree_df.index.values.searchsorted(np.datetime64(date_l))].groupby('sender_id', as_index = False).size()
    old_outdegree = user_info_2.merge(old_outdegree, left_on = 'user_id', right_on = 'sender_id', how= 'left')
    old_outdegree.loc[old_outdegree['size'].isnull(), 'size'] = 0 
    old_outdegree = old_outdegree[['user_id', 'size']].set_index('user_id')
    
    for i in range(df.shape[0]):
        
        if i%interval == 0:
            date_h = df.iloc[i,:]['date_sent']
            new_outdegree = outdegree_df.iloc[outdegree_df.index.values.searchsorted(np.datetime64(date_l)):
                  outdegree_df.index.values.searchsorted(np.datetime64(date_h))].groupby('sender_id', as_index = False).size()
        
            old_outdegree = old_outdegree.add(new_outdegree.set_index('sender_id'), fill_value=0)
            old_outdegree['decile'] = pd.qcut(old_outdegree['size'], 10, labels=False, duplicates='drop') + 1
            old_outdegree['decile'] = old_outdegree[['decile']]  
            date_h = date_l
        
        experiment[str(int(old_outdegree.iloc[i]['decile']))]['Sent'] += 1
        experiment[str(int(old_outdegree.iloc[i]['decile']))]['Reward'] += df.iloc[i, -1]
        Sent = np.array([a[1]['Sent'] for a in experiment.items()])
        Reward = np.array([a[1]['Reward'] for a in experiment.items()])
        
        mean, std = update_1_step(Sent, Reward)
        
        dataset_mean.loc[i, ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = np.transpose(mean) 
        dataset_std.loc[i, ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = np.transpose(std)
        
        print('.', end='')
        
    return dataset_mean, dataset_std

In [44]:
def mab_user(df, outdegree_df, period):
    data_mean = pd.DataFrame()
    data_std = pd.DataFrame()
    for user_id in tqdm(df.user_id.unique()):
        res_mean, res_std = real_time(df[df.user_id == user_id], outdegree_df, period)
        res_mean['user_id'] = user_id
        res_std['user_id'] = user_id
        data_mean = pd.concat([data_mean, res_mean])
        data_std = pd.concat([data_std, res_std])
    return data_mean, data_std

# Real time updating

In [24]:
actions_sent = gen_outbound_creators(follows_sent, shares_sent, likes_sent, comments_sent, tracks,
                                     messages_sent = messages_sent, filter_creators = False)

In [25]:
mask = actions_sent.user_id.isin(tracks_first_month.user_id) 
actions_sent = actions_sent[mask]
actions_sent_filtered = actions_sent.merge(followers_per_creator_60_static, left_on = 'user_id', right_on = 'user_id', how = "inner")

In [26]:
#filter only outbound activities before the follow is received
print('Obtaining fanbase...')
follows_received.columns = ['fan_id', 'user_id', 'date_sent']
followers = follower_list(follows_received)
actions_sent_filtered = actions_sent_filtered.merge(followers, right_on = ['user_id', 'fan_id'],
                                      left_on = ['user_id', 'fan_id'], how = 'left')

mask = (actions_sent_filtered.date_sent < actions_sent_filtered.follower_since) | (actions_sent_filtered.follower_since.isnull())
actions_sent_filtered_non_followers =  actions_sent_filtered[mask]
actions_sent_filtered_non_followers['week_yr_date'] = actions_sent_filtered_non_followers.week_yr.apply(lambda x: process_date(x))

Obtaining fanbase...


/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_90456/1142436393.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_filtered_non_followers['week_yr_date'] = actions_sent_filtered_non_followers.week_yr.apply(lambda x: process_date(x))


In [27]:
import datetime
# Target Creation
delta = datetime.timedelta(days = 7)
mask = (actions_sent_filtered_non_followers['follower_since'] <(actions_sent_filtered_non_followers['date_sent'] + delta))

actions_sent_filtered_non_followers.loc[mask, 'reward'] = 1
mask = actions_sent_filtered_non_followers['reward'].isnull()
actions_sent_filtered_non_followers.loc[mask, 'reward'] = 0

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_90456/2973940690.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_filtered_non_followers.loc[mask, 'reward'] = 1


In [28]:
creators = user_info[user_info.type == 'creator'].user_id.unique()
users = user_info[user_info.type != 'creator'].user_id.unique()

mask = actions_sent_filtered_non_followers.high_low == 'high'
successful_df = actions_sent_filtered_non_followers[mask]

mask = actions_sent_filtered_non_followers.high_low == 'low'
unsuccessful_df = actions_sent_filtered_non_followers[mask]

sucessful_df_cre_cre = successful_df[successful_df.fan_id.isin(creators)]
sucessful_df_cre_use = successful_df[~successful_df.fan_id.isin(creators)]

unsucessful_df_cre_cre = unsuccessful_df[unsuccessful_df.fan_id.isin(creators)]
unsucessful_df_cre_use = unsuccessful_df[~unsuccessful_df.fan_id.isin(creators)]

In [29]:
actions_sent_filtered_non_followers.sort_values(by = 'date_sent', inplace = True)

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_90456/670285413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_filtered_non_followers.sort_values(by = 'date_sent', inplace = True)


In [ ]:
data_sucessful_df_cre_cre_mean, data_sucessful_df_cre_cre_std = mab_user(sucessful_df_cre_cre, data, 20)
data_one_sucessful_df_cre_cre_mean = data_sucessful_df_cre_cre_mean.\
drop_duplicates(subset='user_id', keep='last', inplace=False, ignore_index=False)

data_one_sucessful_df_cre_cre_std = data_sucessful_df_cre_cre_std.\
drop_duplicates(subset='user_id', keep='last', inplace=False, ignore_index=False)

data_sucessful_df_cre_cre_mean.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_sucessful_df_cre_cre_mean.pkl')
data_sucessful_df_cre_cre_std.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_sucessful_df_cre_cre_std.pkl')

data_one_sucessful_df_cre_cre_mean.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_one_sucessful_df_cre_cre_mean.pkl')

data_one_sucessful_df_cre_cre_std.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_one_sucessful_df_cre_cre_std.pkl')

  0%|                                                                                                                                               | 0/626 [00:00<?, ?it/s]

..........................................................................................

  0%|▏                                                                                                                                      | 1/626 [00:04<41:53,  4.02s/it]

...

  0%|▍                                                                                                                                      | 2/626 [00:04<22:54,  2.20s/it]

...

.................................................................................................................................

 16%|█████████████████████▋                                                                                                               | 102/626 [15:30<58:04,  6.65s/it]

........................................

 16%|█████████████████████▉                                                                                                               | 103/626 [15:34<51:09,  5.87s/it]

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
#data_one_sucessful_df_cre_cre[[
#'trials', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'    
#]].mean()

In [ ]:
data_sucessful_df_cre_use_mean, data_sucessful_df_cre_use_std = mab_user(sucessful_df_cre_use, data, 10)
data_one_sucessful_df_cre_use_mean = data_sucessful_df_cre_use_mean.\
drop_duplicates(subset='user_id', keep='last', inplace=False, ignore_index=False)

data_one_sucessful_df_cre_use_std = data_sucessful_df_cre_use_std.\
drop_duplicates(subset='user_id', keep='last', inplace=False, ignore_index=False)

data_sucessful_df_cre_use_mean.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_sucessful_df_cre_use_mean.pkl')
data_sucessful_df_cre_use_std.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_sucessful_df_cre_use_std.pkl')

data_one_sucessful_df_cre_use_mean.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_one_sucessful_df_cre_use_mean.pkl')

data_one_sucessful_df_cre_use_std.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_one_sucessful_df_cre_use_std.pkl')

In [ ]:
#data_one_sucessful_df_cre_use[[
#'trials', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'    
#]].mean()

In [ ]:
data_unsucessful_df_cre_cre_mean, data_unsucessful_df_cre_cre_std = mab_user(sucessful_df_cre_cre, data, 10)
data_one_unsucessful_df_cre_cre_mean = data_unsucessful_df_cre_cre_mean.\
drop_duplicates(subset='user_id', keep='last', inplace=False, ignore_index=False)

data_one_unsucessful_df_cre_cre_std = data_sucessful_df_cre_cre_std.\
drop_duplicates(subset='user_id', keep='last', inplace=False, ignore_index=False)

data_unsucessful_df_cre_cre_mean.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_unsucessful_df_cre_cre_mean.pkl')
data_unsucessful_df_cre_cre_std.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_unsucessful_df_cre_cre_std.pkl')

data_one_unsucessful_df_cre_cre_mean.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_one_unsucessful_df_cre_cre_mean.pkl')

data_one_unsucessful_df_cre_cre_std.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_one_unsucessful_df_cre_cre_std.pkl')

In [ ]:
#data_one_unsucessful_df_cre_cre[[
#'trials', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'    
#]].mean()

In [ ]:
data_unsucessful_df_cre_use_mean, data_unsucessful_df_cre_use_std = mab_user(unsucessful_df_cre_use, data, 10)
data_one_unsucessful_df_cre_use_mean = data_unsucessful_df_cre_use_mean.\
drop_duplicates(subset='user_id', keep='last', inplace=False, ignore_index=False)

data_one_unsucessful_df_cre_use_std = data_unsucessful_df_cre_use_std.\
drop_duplicates(subset='user_id', keep='last', inplace=False, ignore_index=False)

data_unsucessful_df_cre_use_mean.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_unsucessful_df_cre_use_mean.pkl')
data_unsucessful_df_cre_use_std.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_unsucessful_df_cre_use_std.pkl')

data_one_unsucessful_df_cre_use_mean.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_one_unsucessful_df_cre_use_mean.pkl')

data_one_unsucessful_df_cre_use_std.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/data_one_unsucessful_df_cre_use_std.pkl')

In [ ]:
#data_one_unsucessful_df_cre_use[[
#'trials', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'    
#]].mean()

#to do: change code so each interaction updates a creator's Mab
1. class mab? Dict?

In [ ]:
actions_sent_non_followers

In [7]:
user_info_2

,user_id,type,created_at
0,706,creator,2007-11-20 19:23:00
1,845,creator,2007-12-03 17:39:37
2,1255,creator,2008-01-25 21:08:47
3,1506,creator,2008-02-21 18:04:11
4,2667,creator,2008-05-25 18:36:09
...,...,...,...
670741,160366917,,2015-06-29 03:51:08
670742,160556475,,2015-06-30 10:02:16
670743,160592014,,2015-06-30 15:48:27
670744,160615335,,2015-06-30 18:37:51
